In [22]:
# Importing Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from scipy.stats import uniform, randint
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit 
import optuna
from optuna.integration import XGBoostPruningCallback
import shap 
import json
import platform
import pkg_resources
import os
from datetime import datetime


In [23]:
# Load dataset 
df = pd.read_csv('smart_grid_data.csv')
display(df.head())

,Unnamed: 0,Timestamp,Hour of Day,Day of Week,Is Weekend,Season,Month,Historical Electricity Load (kW),Electricity Load,Solar PV Output (kW),...,Curtailment Event Flag,Curtailment Risk / Surplus Flag,Battery State of Charge (SOC) (%),Temperature (°C),Humidity (%),Solar Irradiance (W/m²),Cloud Cover (%),Wind Speed (m/s),Atmospheric Pressure (hPa),Net Load
0,0,2021-01-01 00:00:00+00:00,0,4,0,1,1,192.976646,209.578166,26.711357,...,0,0,46.854540,27.751690,26.752728,298.548493,66.808656,9.210551,1016.729989,92.910873
1,1,2021-01-01 00:30:00+00:00,0,4,0,2,1,709.417292,711.329436,9.158357,...,1,0,34.297665,36.734599,5.300440,18.781034,41.478383,0.193131,1016.800076,670.726196
2,2,2021-01-01 01:00:00+00:00,1,4,0,2,1,441.323762,434.571941,28.972804,...,0,0,28.056109,9.081235,39.869765,73.679871,58.029436,2.719806,1026.064324,389.336376
3,3,2021-01-01 01:30:00+00:00,1,4,0,1,1,110.061053,119.913430,50.747398,...,0,1,73.201450,8.856865,24.710236,110.349077,61.016343,0.093134,1012.320187,-96.219595
4,4,2021-01-01 02:00:00+00:00,2,4,0,2,1,741.130717,747.746618,12.410062,...,0,0,66.116608,13.782261,12.330477,605.363264,26.748752,7.813270,1018.686435,427.137399


In [24]:
# Transform dataframe into datetime series 
# Drop column 
df = df.drop('Unnamed: 0', axis = 1) 
df = df.drop('Season', axis=1) 
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72960 entries, 2021-01-01 00:00:00+00:00 to 2025-02-28 23:30:00+00:00
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Hour of Day                        72960 non-null  int64  
 1   Day of Week                        72960 non-null  int64  
 2   Is Weekend                         72960 non-null  int64  
 3   Month                              72960 non-null  int64  
 4   Historical Electricity Load (kW)   72960 non-null  float64
 5   Electricity Load                   72960 non-null  float64
 6   Solar PV Output (kW)               72960 non-null  float64
 7   Wind Power Output (kW)             72960 non-null  float64
 8   Renewable Forecast Error           72960 non-null  float64
 9   Curtailment Event Flag             72960 non-null  int64  
 10  Curtailment Risk / Surplus Flag    72960 non-null  int64  
 11  Battery

In [25]:
# Rename columns 

df.columns = ["Hour", "Day", "Weekend", "Month", "Hist_Load", "Elec_Load", 
    "Solar_kw", "Wind_kw", "RF_Error", "C_Flag", "C_R_S", "B_SOC", "Temp", 
    "Humidity", "S_Irr", "Cloud", "W_Speed", "HPa", "Net_Load"
    ] 

df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72960 entries, 2021-01-01 00:00:00+00:00 to 2025-02-28 23:30:00+00:00
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Hour       72960 non-null  int64  
 1   Day        72960 non-null  int64  
 2   Weekend    72960 non-null  int64  
 3   Month      72960 non-null  int64  
 4   Hist_Load  72960 non-null  float64
 5   Elec_Load  72960 non-null  float64
 6   Solar_kw   72960 non-null  float64
 7   Wind_kw    72960 non-null  float64
 8   RF_Error   72960 non-null  float64
 9   C_Flag     72960 non-null  int64  
 10  C_R_S      72960 non-null  int64  
 11  B_SOC      72960 non-null  float64
 12  Temp       72960 non-null  float64
 13  Humidity   72960 non-null  float64
 14  S_Irr      72960 non-null  float64
 15  Cloud      72960 non-null  float64
 16  W_Speed    72960 non-null  float64
 17  HPa        72960 non-null  float64
 18  Net_Load   72960 non-null  float64
dtyp

In [ ]:
# 1. ---- DEFINE TARGET VARIABLE ----
# Get the ramp
df['Hist_Ramp'] = df['Net_Load'].diff()

# Get future ramp 
df['Next_Ramp'] = df['Hist_Ramp'].shift(-1)

# 3. Drop the last row because we don't know the "next" ramp for it.
df.dropna(inplace=True)

df.info()

In [27]:
# 1. ---- CALCULATE LAGS AND ROLLING WINDOW STATS ----

# Create Lag Columns
lw_cols = [
    "Hist_Load", "Elec_Load", "Solar_kw", "Wind_kw", "RF_Error", "C_Flag",
    "C_R_S", "B_SOC", "Temp", "Humidity", "S_Irr", "Cloud", "W_Speed", 
    "HPa", "Net_Load", 'Hist_Ramp'
]

# Define Lag durations
lags = [1, 2, 24, 48]

# Define Window Sizes
windows = [3, 6, 12, 24, 48]

# Stats for rolling windows
stats=['mean','std','min','max']

# Function to create lag features
def create_lag_features(df, columns, lags):
    """
    Create lag features for multiple columns.

    Parameters:
    df (pd.DataFrame): Input dataframe
    columns (list): List of column names to create lags for
    lags (list): List of integer lag values (e.g., [1, 2, 48])

    Returns:
    pd.DataFrame: Dataframe with lag features added
    """
    df = df.copy()

    for col in lw_cols:
        # Create lag features
        for lag in lags:
            df[f"{col}_lag_{lag}"] = df[col].shift(lag)

    return df 

def create_window_features(df, columns, windows, stats):
    """
    Create window features for multiple columns.

    Parameters:
    df (pd.DataFrame): Input dataframe
    columns (list): List of column names to create lags for
    lags (list): List of integer lag values (e.g., [1, 2, 48])

    Returns:
    pd.DataFrame: Dataframe with lag features added
    """
    df = df.copy()

    for col in lw_cols:
        # Create window features
        for w in windows:
          for stat in stats:
            df[f"{col}_rw_{stat}"] = df[col].rolling(w).agg(stat)

    return df

In [28]:
# Run Function
df = create_lag_features(df, lw_cols, lags)
df = create_window_features(df, lw_cols, windows, stats)
df = df.dropna()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72910 entries, 2021-01-02 00:30:00+00:00 to 2025-02-28 23:00:00+00:00
Columns: 149 entries, Hour to Hist_Ramp_rw_max
dtypes: float64(143), int64(6)
memory usage: 83.4 MB
None


In [29]:
# 2. ---- CREATE INTERACTION AND CYCLICAL FEATURES ----

period_hour = 24
period_week = 7
period_month=12

# Define Peak Hour Interaction & Temperature Interaction
df['IsPeakHour'] = ((df['Hour'] >= 7) & (df['Hour'] < 22)).astype(int)
df['Temp_pkhr'] = df['Temp'] * df['IsPeakHour']

# Define Seasons and Temperature Interaction
df['IsSummer'] = ((df['Month'] >= 6) & (df['Month'] <= 8)).astype(int)
df['Temp_x_Sum'] = df['Temp'] * df['IsSummer']
df['IsWinter'] = ((df['Month'] == 12) | (df['Month'] <= 2)).astype(int)
df['Temp_x_Win'] = df['Temp'] * df['IsWinter']

# Calculate Cyclical Features
df['hr_sin'] = np.sin(2 * np.pi * df['Hour'] / period_hour)
df['hr_cos'] = np.cos(2 * np.pi * df['Hour'] / period_hour)
df['wk_sin'] = np.sin(2 * np.pi * df['Day'] / period_week)
df['wk_cos'] = np.cos(2 * np.pi * df['Day'] / period_week)
df['mo_sin'] = np.sin(2 * np.pi * df['Month'] / period_month)
df['mo_cos'] = np.cos(2 * np.pi * df['Month'] / period_month)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72910 entries, 2021-01-02 00:30:00+00:00 to 2025-02-28 23:00:00+00:00
Columns: 161 entries, Hour to mo_cos
dtypes: float64(152), int64(9)
memory usage: 90.1 MB
None


In [30]:
print(list(df.columns))
print(len(df.columns))

['Hour', 'Day', 'Weekend', 'Month', 'Hist_Load', 'Elec_Load', 'Solar_kw', 'Wind_kw', 'RF_Error', 'C_Flag', 'C_R_S', 'B_SOC', 'Temp', 'Humidity', 'S_Irr', 'Cloud', 'W_Speed', 'HPa', 'Net_Load', 'Hist_Ramp', 'Next_Ramp', 'Hist_Load_lag_1', 'Hist_Load_lag_2', 'Hist_Load_lag_24', 'Hist_Load_lag_48', 'Elec_Load_lag_1', 'Elec_Load_lag_2', 'Elec_Load_lag_24', 'Elec_Load_lag_48', 'Solar_kw_lag_1', 'Solar_kw_lag_2', 'Solar_kw_lag_24', 'Solar_kw_lag_48', 'Wind_kw_lag_1', 'Wind_kw_lag_2', 'Wind_kw_lag_24', 'Wind_kw_lag_48', 'RF_Error_lag_1', 'RF_Error_lag_2', 'RF_Error_lag_24', 'RF_Error_lag_48', 'C_Flag_lag_1', 'C_Flag_lag_2', 'C_Flag_lag_24', 'C_Flag_lag_48', 'C_R_S_lag_1', 'C_R_S_lag_2', 'C_R_S_lag_24', 'C_R_S_lag_48', 'B_SOC_lag_1', 'B_SOC_lag_2', 'B_SOC_lag_24', 'B_SOC_lag_48', 'Temp_lag_1', 'Temp_lag_2', 'Temp_lag_24', 'Temp_lag_48', 'Humidity_lag_1', 'Humidity_lag_2', 'Humidity_lag_24', 'Humidity_lag_48', 'S_Irr_lag_1', 'S_Irr_lag_2', 'S_Irr_lag_24', 'S_Irr_lag_48', 'Cloud_lag_1', 'Cloud_l

In [ ]:
['Hour', 'Day', 'Weekend', 'Month', 'Hist_Load', 'Elec_Load', 'Solar_kw', 'Wind_kw', 'RF_Error', 'C_Flag', 
'C_R_S', 'B_SOC', 'Temp', 'Humidity', 'S_Irr', 'Cloud', 'W_Speed', 'HPa', 'Net_Load', 'Hist_Ramp', 'Next_Ramp', 
'Hist_Load_lag_1', 'Hist_Load_lag_2', 'Hist_Load_lag_24', 'Hist_Load_lag_48', 'Elec_Load_lag_1', 'Elec_Load_lag_2', 
'Elec_Load_lag_24', 'Elec_Load_lag_48', 'Solar_kw_lag_1', 'Solar_kw_lag_2', 'Solar_kw_lag_24', 'Solar_kw_lag_48', 
'Wind_kw_lag_1', 'Wind_kw_lag_2', 'Wind_kw_lag_24', 'Wind_kw_lag_48', 'RF_Error_lag_1', 'RF_Error_lag_2', 'RF_Error_lag_24', 
'RF_Error_lag_48', 'C_Flag_lag_1', 'C_Flag_lag_2', 'C_Flag_lag_24', 'C_Flag_lag_48', 'C_R_S_lag_1', 'C_R_S_lag_2', 'C_R_S_lag_24', 
'C_R_S_lag_48', 'B_SOC_lag_1', 'B_SOC_lag_2', 'B_SOC_lag_24', 'B_SOC_lag_48', 'Temp_lag_1', 'Temp_lag_2', 'Temp_lag_24', 
'Temp_lag_48', 'Humidity_lag_1', 'Humidity_lag_2', 'Humidity_lag_24', 'Humidity_lag_48', 'S_Irr_lag_1', 'S_Irr_lag_2', 
'S_Irr_lag_24', 'S_Irr_lag_48', 'Cloud_lag_1', 'Cloud_lag_2', 'Cloud_lag_24', 'Cloud_lag_48', 'W_Speed_lag_1', 'W_Speed_lag_2', 
'W_Speed_lag_24', 'W_Speed_lag_48', 'HPa_lag_1', 'HPa_lag_2', 'HPa_lag_24', 'HPa_lag_48', 'Net_Load_lag_1', 'Net_Load_lag_2', 
'Net_Load_lag_24', 'Net_Load_lag_48', 'Hist_Ramp_lag_1', 'Hist_Ramp_lag_2', 'Hist_Ramp_lag_24', 'Hist_Ramp_lag_48', 
'Hist_Load_rw_mean', 'Hist_Load_rw_std', 'Hist_Load_rw_min', 'Hist_Load_rw_max', 'Elec_Load_rw_mean', 'Elec_Load_rw_std', 
'Elec_Load_rw_min', 'Elec_Load_rw_max', 'Solar_kw_rw_mean', 'Solar_kw_rw_std', 'Solar_kw_rw_min', 'Solar_kw_rw_max', 
'Wind_kw_rw_mean', 'Wind_kw_rw_std', 'Wind_kw_rw_min', 'Wind_kw_rw_max', 'RF_Error_rw_mean', 'RF_Error_rw_std', 'RF_Error_rw_min', 
'RF_Error_rw_max', 'C_Flag_rw_mean', 'C_Flag_rw_std', 'C_Flag_rw_min', 'C_Flag_rw_max', 'C_R_S_rw_mean', 'C_R_S_rw_std', 
'C_R_S_rw_min', 'C_R_S_rw_max', 'B_SOC_rw_mean', 'B_SOC_rw_std', 'B_SOC_rw_min', 'B_SOC_rw_max', 'Temp_rw_mean', 'Temp_rw_std', 
'Temp_rw_min', 'Temp_rw_max', 'Humidity_rw_mean', 'Humidity_rw_std', 'Humidity_rw_min', 'Humidity_rw_max', 'S_Irr_rw_mean', 
'S_Irr_rw_std', 'S_Irr_rw_min', 'S_Irr_rw_max', 'Cloud_rw_mean', 'Cloud_rw_std', 'Cloud_rw_min', 'Cloud_rw_max', 'W_Speed_rw_mean', 
'W_Speed_rw_std', 'W_Speed_rw_min', 'W_Speed_rw_max', 'HPa_rw_mean', 'HPa_rw_std', 'HPa_rw_min', 'HPa_rw_max', 'Net_Load_rw_mean', 
'Net_Load_rw_std', 'Net_Load_rw_min', 'Net_Load_rw_max', 'Hist_Ramp_rw_mean', 'Hist_Ramp_rw_std', 'Hist_Ramp_rw_min', 
'Hist_Ramp_rw_max', 'IsPeakHour', 'Temp_pkhr', 'IsSummer', 'Temp_x_Sum', 'IsWinter', 'Temp_x_Win', 'hr_sin', 'hr_cos', 
'wk_sin', 'wk_cos', 'mo_sin', 'mo_cos']

In [31]:
# Verify dates 
print(df.index[0])  # First Date
print(df.index[-1]) # Last date
print(len(df))    # Length of DF

2021-01-02 00:30:00+00:00
2025-02-28 23:00:00+00:00
72910


In [32]:
# Drop and Split

train_df = df.loc['2021-01-02 00:30':'2023-12-23 23:30']
X_train = train_df.drop(columns=['Next_Ramp'])
y_train = train_df['Next_Ramp']

val_df = df.loc['2024-01-01 00:00':'2024-06-30 23:30']
X_val = val_df.drop(columns=['Next_Ramp'])
y_val = val_df['Next_Ramp']

test_df = df.loc['2024-07-01 00:00':'2024-12-31 23:30']
X_test = test_df.drop(columns=['Next_Ramp'])
y_test = test_df['Next_Ramp']


In [33]:
# Create DMatrices
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dval = xgb.DMatrix(data=X_val, label=y_val)
dtest = xgb.DMatrix(data=X_test, label=y_test)

In [34]:
# Defining and running Baseline Model
base_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=10, seed=126)

# Training the model
base_model.fit(X_train, y_train)

base_pred = base_model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, base_pred))
print(f"Baseline RMSE: {rmse}")


Baseline RMSE: 277.6446480189662


In [35]:
# Random Search 
# Define the base estimator
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Create DMatrix for training data
elec_dmatrix = xgb.DMatrix(data=X_train, label=y_train)

# Defining Hyperparaemeter Search Space
param_dist = {
    'n_estimators' : randint(100, 1000), # No of boosting rounds
    'max_depth' : randint(3, 10), # Max depth of each tree
    'learning_rate' : uniform(0.01, 0.3), # Learning rate
    'subsample' : uniform(0.5, 0.5), # Fraction of samples per tree
    'colsample_bytree' : uniform(0.6, 0.4), # #Fraction of features per tree
    'gamma' : uniform(0, 0.5) # Minimum loss reduction per tree
}

# Setup Time Series Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Initialize RandomSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=10,
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    verbose=1,
    n_jobs=-1,
    random_state = 42
)

In [36]:
# Fit the model
random_search.fit(X_train, y_train) 

# Print the best parameters
print("Best Parameters:", random_search.best_params_)

# Get the best model
best_model = random_search.best_estimator_ 

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters: {'colsample_bytree': np.float64(0.8721230154351118), 'gamma': np.float64(0.2252496259847715), 'learning_rate': np.float64(0.013979488347959958), 'max_depth': 3, 'n_estimators': 415, 'subsample': np.float64(0.7816441089227697)}


In [37]:
# Fit the model
best_model.fit(X_train, y_train)

# Make predictions
tuned_pred = best_model.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, tuned_pred))
print(f'RMSE: {rmse}')  

RMSE: 277.27598026390916


In [38]:
# Save the model
best_model.save_model("xgboost_smart_ml.ubj")

In [40]:
# Feature Names 
print(best_model.get_booster().feature_names)

['Hour', 'Day', 'Weekend', 'Month', 'Hist_Load', 'Elec_Load', 'Solar_kw', 'Wind_kw', 'RF_Error', 'C_Flag', 'C_R_S', 'B_SOC', 'Temp', 'Humidity', 'S_Irr', 'Cloud', 'W_Speed', 'HPa', 'Net_Load', 'Hist_Ramp', 'Hist_Load_lag_1', 'Hist_Load_lag_2', 'Hist_Load_lag_24', 'Hist_Load_lag_48', 'Elec_Load_lag_1', 'Elec_Load_lag_2', 'Elec_Load_lag_24', 'Elec_Load_lag_48', 'Solar_kw_lag_1', 'Solar_kw_lag_2', 'Solar_kw_lag_24', 'Solar_kw_lag_48', 'Wind_kw_lag_1', 'Wind_kw_lag_2', 'Wind_kw_lag_24', 'Wind_kw_lag_48', 'RF_Error_lag_1', 'RF_Error_lag_2', 'RF_Error_lag_24', 'RF_Error_lag_48', 'C_Flag_lag_1', 'C_Flag_lag_2', 'C_Flag_lag_24', 'C_Flag_lag_48', 'C_R_S_lag_1', 'C_R_S_lag_2', 'C_R_S_lag_24', 'C_R_S_lag_48', 'B_SOC_lag_1', 'B_SOC_lag_2', 'B_SOC_lag_24', 'B_SOC_lag_48', 'Temp_lag_1', 'Temp_lag_2', 'Temp_lag_24', 'Temp_lag_48', 'Humidity_lag_1', 'Humidity_lag_2', 'Humidity_lag_24', 'Humidity_lag_48', 'S_Irr_lag_1', 'S_Irr_lag_2', 'S_Irr_lag_24', 'S_Irr_lag_48', 'Cloud_lag_1', 'Cloud_lag_2', 'Cloud